##### Model user’s skipping behavior: Build and estimate (or train) a model on the first 15,000 observations in the data (thetraining and validation sample) that provides the most accurate and robust predictionsof user skipping behavior on the last 5,000 observationsin the data (the hold-out sample).

In [57]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as spst
from scipy.optimize import minimize
import scipy.special as spsp
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_roc_curve
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score


In [35]:
data=pd.read_csv('spotify.csv')
data['early']=data['early'].apply(lambda x: int(x))
data['late']=data['late'].apply(lambda x: int(x))
data
df=data.copy()
df['skip_speed']=[df['early'][i]+df['late'][i] for i in range(df.shape[0])]
df.drop(['early','late'],axis=1,inplace=True)

In [63]:
y=df.iloc[:,-1]
x=df.iloc[:,:-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=40,shuffle=True)


In [64]:
######### Fitting scaler on train data #########
scaler=StandardScaler()
scaler.fit(x_train)


######## Scaling both train and test #########
x_train= scaler.transform(x_train)
x_test= scaler.transform(x_test)

In [75]:
scaler2=StandardScaler()
scaler2.fit(np.array([y_train]))
y_=np.array([y_train])
y_=scaler2.transform(y_)

In [79]:
y_.shape

(1, 15000)